In [3]:
import os
import copy
import random
import time

import graphviz
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import wandb

import optuna
from optuna.samplers import TPESampler

from scipy.optimize import minimize
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')

seed = 42
random.seed(seed)
np.random.seed(seed)
sampler = TPESampler(seed=seed)

In [4]:
save_vis = True
output_dir = 'models_media'
if save_vis:
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

In [5]:
# Прочитаем датасет
dataset = pd.read_csv(
    '/home/rito4ka/dev/magistratura/miptml_projects/songs_classification/spotify-2023.csv',
    encoding='latin-1'
)

# Добавим категориальные коды для признаков
dataset['key'] = pd.Categorical(dataset['key'])
dataset['key'] = dataset['key'].cat.codes
dataset['mode'] = pd.Categorical(dataset['mode'])
dataset['mode'] = dataset['mode'].cat.codes

# Скастуем к инту то, что имело тип строки
dataset['in_deezer_playlists'] = [int(item.replace(',', '')) for item in dataset['in_deezer_playlists']]
dataset['in_shazam_charts'] = [int(item.replace(',', '')) if isinstance(item, str) else item
                               for item in dataset['in_shazam_charts']]

# Нагенерим столбец с целевым признаком
dataset['is_in_spotify_charts'] = [int(bool(item)) for item in dataset['in_spotify_charts']]

# Уберём лишние столбцы (Название трека и имя артиста; присутствие в плейлистах и чартах
# на платформах кроме Spotify, на Spotify скипнем только чарты, тк это целевой признак)
dataset = dataset.drop(
    ['artist(s)_name', 'in_spotify_charts', 'in_apple_playlists', 'in_apple_charts',
     'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts'], axis=1
)

dataset.head()

,track_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,is_in_spotify_charts
0,Seven (feat. Latto) (Explicit Ver.),2,2023,7,14,553,141381703.0,125,2,0,80,89,83,31,0,8,4,1
1,LALA,1,2023,3,23,1474,133716286.0,92,3,0,71,61,74,7,0,10,4,1
2,vampire,1,2023,6,30,1397,140003974.0,138,7,0,51,32,53,17,0,31,6,1
3,Cruel Summer,1,2019,8,23,7858,800840817.0,170,0,0,55,58,72,11,0,11,15,1
4,WHERE SHE GOES,1,2023,5,18,3133,303236322.0,144,0,1,65,23,80,14,63,11,6,1


In [6]:
# Уберём имя песни и разделим на train и test выборки.
dataset = dataset.drop('track_name', axis=1)

# Обработаем nan
dataset = dataset.fillna(0)

X = dataset.drop('is_in_spotify_charts', axis=1)
y = dataset['is_in_spotify_charts']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

X_train_original = copy.deepcopy(X_train)
X_test_original = copy.deepcopy(X_test)

# Произведём масштабирование:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Обучение LogReg**

In [5]:
# Соберём конфиг для grid search с wandb sweep и инициализируем

sweep_config = {
    'name': 'logistic_regression_sweep',
    'method': 'grid',
    'parameters': {
        'C': {'values': [0.001, 0.01, 0.1, 1.0]},  # Параметр обратной силы регуляризации
        'solver': {'values': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']},  # Алгоритм оптимизации
        'class_weight': {'values': [None, 'balanced']},  # Веса классов
        'max_iter': {'values': [10, 42, 100]},
    },
    'metric': {'goal': 'maximize', 'name': 'accuracy'}
}

# sweep_id = wandb.sweep(sweep_config, project='miptml_songs')

In [6]:
def search_logreg(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        C = config.C
        solver = config.solver
        class_weight = config.class_weight
        max_iter = config.max_iter
        
        model = LogisticRegression(
            penalty='l2',
            class_weight=class_weight,
            C=C,
            solver=solver,
            max_iter=max_iter,
            random_state=seed
        )
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)

        wandb.log({"accuracy": accuracy})

In [7]:
# wandb.agent(sweep_id, search_logreg)

**Оптимизируем С с помощью optuna**

In [8]:
best_params = {
    'penalty' : 'l2',
    'class_weight' : None,
    'solver' : 'saga',
    'max_iter' : 42,
}

In [1]:
def objective(trial):    
    C = trial.suggest_float('C', 0.0001, 1.0)
    
    model = LogisticRegression(
            penalty=best_params['penalty'],
            class_weight=best_params['class_weight'],
            C=C,
            solver=best_params['solver'],
            max_iter=best_params['max_iter'],
            random_state=seed
        )
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [2]:
study = optuna.create_study(direction="maximize", sampler=sampler) 
study.optimize(objective, show_progress_bar=True)

NameError: name 'optuna' is not defined

In [11]:
best_C_logreg = study.best_params['C']
best_C_logreg

0.0017629478293904322

In [12]:
model = LogisticRegression(
            penalty=best_params['penalty'],
            class_weight=best_params['class_weight'],
            C=best_C_logreg,
            solver=best_params['solver'],
            max_iter=best_params['max_iter'],
            random_state=seed
        )

scores = cross_val_score(model, X_train, y_train, cv=5)
print(f'mean cross-val = {np.mean(scores)}')

mean cross-val = 0.5905744754041968


In [13]:
model = LogisticRegression(
            penalty=best_params['penalty'],
            class_weight=best_params['class_weight'],
            C=best_C_logreg,
            solver=best_params['solver'],
            max_iter=best_params['max_iter'],
            random_state=seed
        )
model.fit(X_train, y_train)
        
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1:.3f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Accuracy: 0.649
F1 score: 0.774
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.12      0.21        72
           1       0.65      0.97      0.77       119

    accuracy                           0.65       191
   macro avg       0.67      0.55      0.49       191
weighted avg       0.66      0.65      0.56       191



In [14]:
current_time = time.time()
for x in X_train:
    model.predict(x.reshape(1, -1))
passed_time = time.time() - current_time
print(f'inference time: {passed_time / len(X_train)}')

inference time: 6.0524840367434845e-05


**Обучение SVM**

In [15]:
# sweep_config = {
#     'name': 'support_vector_machine_sweep_',
#     'method': 'grid',
#     'parameters': {
#         'C': {'values': [0.001, 0.01, 0.1, 1.0]},
#         'class_weight': {'values': [None, 'balanced']},
#         'kernel': {'values': ['linear', 'poly', 'rbf', 'sigmoid']},
#         'max_iter': {'values': [42, 70, 100]},
#     },
#     'metric': {'goal': 'maximize', 'name': 'accuracy'}
# }

sweep_config = {
    'name': 'support_vector_machine_sweep_poly',
    'method': 'grid',
    'parameters': {
        'C': {'values': [0.001, 0.01, 0.1, 1.0]},
        'degree': {'values': [1, 2, 3, 4]},
        'gamma': {'values': ['scale', 'auto']},
        'coef0': {'values': [0.0, 1.0]},
        'probability' : {'values': [True, False]},
        'max_iter': {'values': [42, 70, 100]},
    },
    'metric': {'goal': 'maximize', 'name': 'accuracy'}
}

# sweep_id = wandb.sweep(sweep_config, project='miptml_songs')

In [16]:
def search_svm(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        C = config.C
        degree = config.degree
        gamma = config.gamma
        coef0 = config.coef0
        probability = config.probability
        max_iter = config.max_iter
        
        model = SVC(
            class_weight=None,
            C=C,
            degree=degree,
            gamma=gamma,
            coef0=coef0,
            probability=probability,
            kernel='poly',
            max_iter=max_iter,
            random_state=seed
        )
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)

        wandb.log({"accuracy": accuracy})

In [17]:
# wandb.agent(sweep_id, search_svm)

**Оптимизируем параметры с помощью optuna**

In [7]:
best_params = {
    'class_weight': None,
    'C': None, # исследовать
    'degree': 2,
    'gamma': 'scale',
    'coef0': None, # исследовать
    'probability': False,
    'kernel': 'poly',
    'max_iter': None, # исследовать
}

In [8]:
def objective(trial):    
    C = trial.suggest_float('C', 0.0001, 1.0)
    coef0 = trial.suggest_float('coef0', 0.0001, 1.0)
    max_iter = trial.suggest_int('max_iter', 1, 125)
    
    model = SVC(
        class_weight=best_params['class_weight'],
        C=C,
        degree=best_params['degree'],
        gamma=best_params['gamma'],
        coef0=coef0,
        probability=best_params['probability'],
        kernel=best_params['kernel'],
        max_iter=max_iter,
        random_state=seed
    )
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [9]:
study = optuna.create_study(direction="maximize", sampler=sampler) 
study.optimize(objective, show_progress_bar=True, n_trials=5000)

[I 2023-11-15 12:19:51,287] A new study created in memory with name: no-name-214458a4-f6b5-43e1-a041-c7e20db2639b


  0%|          | 0/5000 [00:00<?, ?it/s]

[I 2023-11-15 12:19:51,304] Trial 0 finished with value: 0.6282722513089005 and parameters: {'C': 0.37460266483547777, 'coef0': 0.9507192349792751, 'max_iter': 92}. Best is trial 0 with value: 0.6282722513089005.
[I 2023-11-15 12:19:51,309] Trial 1 finished with value: 0.612565445026178 and parameters: {'C': 0.5986986183486169, 'coef0': 0.15610303857839228, 'max_iter': 20}. Best is trial 0 with value: 0.6282722513089005.
[I 2023-11-15 12:19:51,322] Trial 2 finished with value: 0.6073298429319371 and parameters: {'C': 0.05817780380698265, 'coef0': 0.8661895281603577, 'max_iter': 76}. Best is trial 0 with value: 0.6282722513089005.
[I 2023-11-15 12:19:51,334] Trial 3 finished with value: 0.643979057591623 and parameters: {'C': 0.7081017705382658, 'coef0': 0.020682435846372867, 'max_iter': 122}. Best is trial 3 with value: 0.643979057591623.
[I 2023-11-15 12:19:51,340] Trial 4 finished with value: 0.612565445026178 and parameters: {'C': 0.8324593965363417, 'coef0': 0.21241787676720833, 'm

[I 2023-11-15 12:19:52,148] Trial 48 finished with value: 0.6073298429319371 and parameters: {'C': 0.9515366948669193, 'coef0': 0.786989137616811, 'max_iter': 9}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:52,166] Trial 49 finished with value: 0.6282722513089005 and parameters: {'C': 0.6154806944692243, 'coef0': 0.9658408472112512, 'max_iter': 111}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:52,183] Trial 50 finished with value: 0.6230366492146597 and parameters: {'C': 0.5443558053025389, 'coef0': 0.9552324879024136, 'max_iter': 121}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:52,203] Trial 51 finished with value: 0.6230366492146597 and parameters: {'C': 0.6799527994137278, 'coef0': 0.5145527859844604, 'max_iter': 114}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:52,226] Trial 52 finished with value: 0.6387434554973822 and parameters: {'C': 0.8028863849149711, 'coef0': 0.0827328200445

[I 2023-11-15 12:19:53,010] Trial 94 finished with value: 0.6230366492146597 and parameters: {'C': 0.9482210092926084, 'coef0': 0.8032144640007102, 'max_iter': 46}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:53,039] Trial 95 finished with value: 0.643979057591623 and parameters: {'C': 0.9289061401070975, 'coef0': 0.7498003789025423, 'max_iter': 56}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:53,054] Trial 96 finished with value: 0.6335078534031413 and parameters: {'C': 0.978246171562623, 'coef0': 0.6509523661353669, 'max_iter': 26}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:53,069] Trial 97 finished with value: 0.6335078534031413 and parameters: {'C': 0.9552875998394532, 'coef0': 0.7722565742404548, 'max_iter': 39}. Best is trial 15 with value: 0.6596858638743456.
[I 2023-11-15 12:19:53,087] Trial 98 finished with value: 0.6230366492146597 and parameters: {'C': 0.902319333574908, 'coef0': 0.6995131708615109,

...

[I 2023-11-15 12:24:11,145] Trial 4823 finished with value: 0.6492146596858639 and parameters: {'C': 0.7809241079356871, 'coef0': 0.8423152022597236, 'max_iter': 116}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:11,239] Trial 4824 finished with value: 0.643979057591623 and parameters: {'C': 0.8183765535054867, 'coef0': 0.8351336949043316, 'max_iter': 118}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:11,320] Trial 4825 finished with value: 0.6282722513089005 and parameters: {'C': 0.7685814593983913, 'coef0': 0.8172525578160361, 'max_iter': 114}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:11,419] Trial 4826 finished with value: 0.643979057591623 and parameters: {'C': 0.8024622403151184, 'coef0': 0.8511394171906195, 'max_iter': 112}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:11,500] Trial 4827 finished with value: 0.6230366492146597 and parameters: {'C': 0.8262830361461411, 'coef

[I 2023-11-15 12:24:14,382] Trial 4862 finished with value: 0.6282722513089005 and parameters: {'C': 0.7724910601927489, 'coef0': 0.7985978647488224, 'max_iter': 112}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:14,469] Trial 4863 finished with value: 0.6492146596858639 and parameters: {'C': 0.8418573942655991, 'coef0': 0.83598106702116, 'max_iter': 122}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:14,547] Trial 4864 finished with value: 0.6596858638743456 and parameters: {'C': 0.7975426432991953, 'coef0': 0.7897434606032717, 'max_iter': 119}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:14,629] Trial 4865 finished with value: 0.6492146596858639 and parameters: {'C': 0.7610662605226484, 'coef0': 0.8107372969074841, 'max_iter': 116}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:14,709] Trial 4866 finished with value: 0.6335078534031413 and parameters: {'C': 0.8179275919306672, 'coef

[I 2023-11-15 12:24:18,059] Trial 4902 finished with value: 0.6544502617801047 and parameters: {'C': 0.8488767056412608, 'coef0': 0.8374753172558327, 'max_iter': 114}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:18,176] Trial 4903 finished with value: 0.6282722513089005 and parameters: {'C': 0.35727354763457064, 'coef0': 0.8980707635444638, 'max_iter': 117}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:18,281] Trial 4904 finished with value: 0.6701570680628273 and parameters: {'C': 0.7415154082368195, 'coef0': 0.871883567574546, 'max_iter': 125}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:18,365] Trial 4905 finished with value: 0.6492146596858639 and parameters: {'C': 0.8056821709177306, 'coef0': 0.8108267190177811, 'max_iter': 121}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:18,443] Trial 4906 finished with value: 0.6335078534031413 and parameters: {'C': 0.8536302571654838, 'co

[I 2023-11-15 12:24:21,693] Trial 4940 finished with value: 0.6335078534031413 and parameters: {'C': 0.34006591451068674, 'coef0': 0.4390787690197193, 'max_iter': 122}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:21,797] Trial 4941 finished with value: 0.6230366492146597 and parameters: {'C': 0.2158819828540407, 'coef0': 0.8562340298193116, 'max_iter': 119}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:21,878] Trial 4942 finished with value: 0.6492146596858639 and parameters: {'C': 0.8352490145262158, 'coef0': 0.4305226729390748, 'max_iter': 113}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:21,978] Trial 4943 finished with value: 0.6230366492146597 and parameters: {'C': 0.3874666131820006, 'coef0': 0.8161016058187823, 'max_iter': 123}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:22,063] Trial 4944 finished with value: 0.6335078534031413 and parameters: {'C': 0.7427111980543484, 'c

[I 2023-11-15 12:24:25,160] Trial 4978 finished with value: 0.612565445026178 and parameters: {'C': 0.8332864073601183, 'coef0': 0.6538388323801376, 'max_iter': 123}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:25,243] Trial 4979 finished with value: 0.6282722513089005 and parameters: {'C': 0.45016901535006415, 'coef0': 0.48890558039558935, 'max_iter': 112}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:25,328] Trial 4980 finished with value: 0.643979057591623 and parameters: {'C': 0.783363125936674, 'coef0': 0.7862871603769358, 'max_iter': 117}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:25,413] Trial 4981 finished with value: 0.6073298429319371 and parameters: {'C': 0.7068778756360004, 'coef0': 0.7555751566057094, 'max_iter': 121}. Best is trial 4802 with value: 0.6910994764397905.
[I 2023-11-15 12:24:25,500] Trial 4982 finished with value: 0.6178010471204188 and parameters: {'C': 0.26072131839269674, 'co

In [10]:
best_C, best_coef0, best_max_iter = study.best_params.values()
best_C, best_coef0, best_max_iter

(0.814581315080938, 0.8156256169657048, 115)

In [11]:
model = SVC(
    class_weight=None,
    C=0.814581315080938,
    degree=2,
    gamma='scale',
    coef0=0.8156256169657048,
    probability=False,
    kernel='poly',
    max_iter=115,
    random_state=seed
)

scores = cross_val_score(model, X_train, y_train, cv=10)
print(f'mean cross-val = {np.mean(scores)}')

mean cross-val = 0.5590054682159946


In [12]:
model = SVC(
    class_weight=None,
    C=0.814581315080938,
    degree=2,
    gamma='scale',
    coef0=0.8156256169657048,
    probability=False,
    kernel='poly',
    max_iter=115,
    random_state=seed
)

model.fit(X_train, y_train)
        
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1:.3f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Accuracy: 0.691
F1 score: 0.799
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.21      0.34        72
           1       0.67      0.98      0.80       119

    accuracy                           0.69       191
   macro avg       0.78      0.60      0.57       191
weighted avg       0.75      0.69      0.62       191



In [79]:
current_time = time.time()
for x in X_train:
    model.predict(x.reshape(1, -1))
passed_time = time.time() - current_time
print(f'inference time: {passed_time / len(X_train)}')

inference time: 7.920002374123401e-05


**Обучение дерева**

In [26]:
def objective(trial):    
    criterion = trial.suggest_categorical(
        'criterion', ['gini', 'entropy', 'log_loss'])
    splitter = trial.suggest_categorical(
        'splitter', ['best', 'random'])
    max_depth = trial.suggest_int("max_depth", 2, 17)
    min_samples_split = trial.suggest_int(
        "min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int(
        'min_samples_leaf', 1, 5)
    min_weight_fraction_leaf = trial.suggest_float(
        'min_weight_fraction_leaf', 0.0, 0.5)
    max_features = trial.suggest_categorical(
        'max_features', ['sqrt', 'log2'])
    max_leaf_nodes = trial.suggest_int(
        'max_leaf_nodes', 2, 1024)
    min_impurity_decrease = trial.suggest_float(
        'min_impurity_decrease', 0.0, 1.0)
    
    model = DecisionTreeClassifier(
        criterion=criterion,
        splitter=splitter,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        min_weight_fraction_leaf=min_weight_fraction_leaf,
        max_features=max_features,
        max_leaf_nodes=max_leaf_nodes,
        min_impurity_decrease=min_impurity_decrease,
        class_weight=None,
        random_state=seed)
    model.fit(X_train_original, y_train)
    
    y_pred = model.predict(X_test_original)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [27]:
study = optuna.create_study(direction="maximize", sampler=sampler) 
study.optimize(objective, show_progress_bar=True)

[I 2023-10-31 14:38:50,429] A new study created in memory with name: no-name-e8404377-cd80-426c-9561-b209a48bc669
[I 2023-10-31 14:38:50,439] Trial 0 finished with value: 0.6230366492146597 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 12, 'min_samples_split': 4, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.2733551396716398, 'max_features': 'log2', 'max_leaf_nodes': 794, 'min_impurity_decrease': 0.9394989415641891}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:50,443] Trial 1 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 2, 'min_samples_split': 4, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.13567451588694796, 'max_features': 'sqrt', 'max_leaf_nodes': 289, 'min_impurity_decrease': 0.5426960831582485}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:50,447] Trial 2 finished with value: 0.6230366492146597 and parameters: {'criterion

[I 2023-10-31 14:38:50,730] Trial 21 finished with value: 0.6230366492146597 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.33273932065110434, 'max_features': 'log2', 'max_leaf_nodes': 15, 'min_impurity_decrease': 0.40721579545051795}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:50,756] Trial 22 finished with value: 0.6230366492146597 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 2, 'min_samples_split': 2, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.34900380066221814, 'max_features': 'log2', 'max_leaf_nodes': 169, 'min_impurity_decrease': 0.3394412134896717}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:50,779] Trial 23 finished with value: 0.6230366492146597 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 8, 'min_samples_split': 3, 'min_samples_leaf': 3, 'min_weight_fract

[I 2023-10-31 14:38:51,191] Trial 42 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.008090738932034547, 'max_features': 'sqrt', 'max_leaf_nodes': 630, 'min_impurity_decrease': 0.985409097161033}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:51,215] Trial 43 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.08125465105826056, 'max_features': 'sqrt', 'max_leaf_nodes': 750, 'min_impurity_decrease': 0.7799222597324698}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:51,235] Trial 44 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 2, 'min_w

[I 2023-10-31 14:38:51,688] Trial 63 finished with value: 0.6230366492146597 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.3900433205257687, 'max_features': 'sqrt', 'max_leaf_nodes': 338, 'min_impurity_decrease': 0.5224463780513139}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:51,715] Trial 64 finished with value: 0.6230366492146597 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0.3521593283476974, 'max_features': 'sqrt', 'max_leaf_nodes': 142, 'min_impurity_decrease': 0.9988470894909939}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:51,741] Trial 65 finished with value: 0.6230366492146597 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 2, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf

[I 2023-10-31 14:38:52,250] Trial 84 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 4, 'min_samples_split': 8, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0920464662834811, 'max_features': 'log2', 'max_leaf_nodes': 449, 'min_impurity_decrease': 0.5948714221984043}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:52,284] Trial 85 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.026103448117449392, 'max_features': 'log2', 'max_leaf_nodes': 73, 'min_impurity_decrease': 0.8054559900808361}. Best is trial 0 with value: 0.6230366492146597.
[I 2023-10-31 14:38:52,315] Trial 86 finished with value: 0.6230366492146597 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 2, 'min_weight_fr

...

[I 2023-10-31 14:45:35,654] Trial 3843 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.2424838292113684, 'max_features': 'sqrt', 'max_leaf_nodes': 113, 'min_impurity_decrease': 0.8281743019502522}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:35,906] Trial 3844 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 7, 'min_samples_split': 5, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.11069409670637423, 'max_features': 'sqrt', 'max_leaf_nodes': 139, 'min_impurity_decrease': 0.4012291716762009}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:36,257] Trial 3845 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 5, 'mi

[I 2023-10-31 14:45:40,500] Trial 3864 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.03519915199829138, 'max_features': 'sqrt', 'max_leaf_nodes': 81, 'min_impurity_decrease': 0.2098625067149631}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:40,728] Trial 3865 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0002021878706718657, 'max_features': 'sqrt', 'max_leaf_nodes': 153, 'min_impurity_decrease': 0.04916883748392128}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:40,932] Trial 3866 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 5, 

[I 2023-10-31 14:45:44,497] Trial 3885 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.08087431405707077, 'max_features': 'sqrt', 'max_leaf_nodes': 29, 'min_impurity_decrease': 0.7149005964968556}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:44,683] Trial 3886 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.3890580183167912, 'max_features': 'sqrt', 'max_leaf_nodes': 140, 'min_impurity_decrease': 0.0005362246830330665}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:44,869] Trial 3887 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 5, '

[I 2023-10-31 14:45:48,513] Trial 3906 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.21707472928710175, 'max_features': 'sqrt', 'max_leaf_nodes': 93, 'min_impurity_decrease': 0.01843225368522745}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:48,710] Trial 3907 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.058818551001101796, 'max_features': 'sqrt', 'max_leaf_nodes': 149, 'min_impurity_decrease': 0.503927801394239}. Best is trial 3312 with value: 0.7015706806282722.
[I 2023-10-31 14:45:48,904] Trial 3908 finished with value: 0.6230366492146597 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 5, 'm

KeyboardInterrupt: 

In [28]:
best_tree_params = study.best_params
best_tree_params

{'criterion': 'log_loss',
 'splitter': 'best',
 'max_depth': 9,
 'min_samples_split': 4,
 'min_samples_leaf': 5,
 'min_weight_fraction_leaf': 0.05466435731880885,
 'max_features': 'sqrt',
 'max_leaf_nodes': 103,
 'min_impurity_decrease': 0.00043010223839798253}

In [29]:
best_params = best_tree_params
# best_params = {
#     'criterion': 'log_loss',
#     'splitter': 'best', 'max_depth': 9,
#     'min_samples_split': 6, 'min_samples_leaf': 2,
#     'min_weight_fraction_leaf': 0.0545000258976131,
#     'max_features': 'sqrt', 'max_leaf_nodes': 120,
#     'min_impurity_decrease': 0.0002486540392544447
# }

model = DecisionTreeClassifier(
        criterion=best_params['criterion'],
        splitter=best_params['splitter'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        min_weight_fraction_leaf=best_params['min_weight_fraction_leaf'],
        max_features=best_params['max_features'],
        max_leaf_nodes=best_params['max_leaf_nodes'],
        min_impurity_decrease=best_params['min_impurity_decrease'],
        class_weight=None,
        random_state=seed)

model.fit(X_train_original, y_train)


y_pred = model.predict(X_test_original)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1:.3f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Accuracy: 0.702
F1 score: 0.765
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.57      0.59        72
           1       0.75      0.78      0.77       119

    accuracy                           0.70       191
   macro avg       0.68      0.68      0.68       191
weighted avg       0.70      0.70      0.70       191



In [30]:
current_time = time.time()
for x in X_train:
    model.predict(x.reshape(1, -1))
passed_time = time.time() - current_time
print(f'inference time: {passed_time / len(X_train)}')

inference time: 6.556229328545998e-05


In [31]:
dot_data = tree.export_graphviz(model, out_file=None,
                                filled=True, rounded=True,
                                special_characters=True)  

graph = graphviz.Source(dot_data)
graph.render('songs_decision_tree')

'songs_decision_tree.pdf'

**Baseline**

In [32]:
class nicenessModel:
    def __init__(self, boarder:float=52.65):
        self.boarder = boarder
        
    def quality_function(self, energy:int, released_year:int) -> float:
        return (energy / 100) * (released_year - 1900)
    
    def __call__(self, instance:dict):
        if len(set(['energy_%', 'released_year']).intersection(set(instance.keys()))) != 2:
            raise ValueError('bad instance')        
        quality = self.quality_function(instance['energy_%'], instance['released_year'])
        return int(quality >= self.boarder)

In [33]:
baseline = nicenessModel()
current_time = time.time()
baseline_pred = [baseline(dict(row)) for index, row in X_test_original.iterrows()]
passed_time = time.time() - current_time
print(f'inference time: {passed_time / len(X_test_original)}')
    
accuracy = accuracy_score(y_test, baseline_pred)
print(f'Accuracy: {accuracy:.3f}')

f1 = f1_score(y_test, baseline_pred)
print(f'F1 score: {f1:.3f}')

report = classification_report(y_test, baseline_pred)
print('Classification Report:\n', report)

inference time: 9.764052186336817e-05
Accuracy: 0.623
F1 score: 0.745
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.19      0.28        72
           1       0.64      0.88      0.74       119

    accuracy                           0.62       191
   macro avg       0.57      0.54      0.51       191
weighted avg       0.59      0.62      0.57       191

